In [21]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
from numpy import int16
from enum import Enum

In [32]:
# Using enum class create enumerations
class Constante(Enum):
   march_1_2021 = "2021-03-01"
   specific = "2021-06-21"

In [33]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = f'''
    SELECT 
    agent_username, COUNT(*) AS nbre,
    dsd.organisation,
    dsd.created_ad as entry_date
FROM
    caris_db.dreams_surveys_data dsd
WHERE
    dsd.created_ad >= '{Constante.specific.value}'
GROUP BY agent_username
'''

agent = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

## Agent DREAMS sortant du Systeme

In [34]:
agent.rename(columns={'agent_username':'username'},inplace=True)
agent

,username,nbre,organisation,entry_date
0,1yvrose,22,CARIS,2021-06-25 05:45:47
1,2ernst,43,CARIS,2021-06-24 08:11:54
2,6francenel,30,CARIS,2021-06-23 15:40:37
3,6frazyouseline,2,CARIS,2021-06-28 09:02:32
4,6lovemyschinider,28,CARIS,2021-06-25 19:21:23
5,6paulina,85,CARIS,2021-06-23 12:59:38


In [35]:
agent_name = pd.read_excel("caris-test_users_7453177b42914ad2978e3c465e39dab1.xlsx")

In [36]:
agent_name = agent_name[["username","name"]]

## Liste de tous les agents avec Leur nom

In [37]:
agent_name

,username,name
0,1aldonzar,elcie aldonzar
1,1alexandra,Alexandra Joseph
2,1annuelle,Annuelle Eustache
3,1claudius,Claudius Gladys
4,1coldina,Coldina Duverny
...,...,...
742,s2julnor,Julnor Beaubrun
743,s2mirlene,Vernet Mirlene
744,stfleur,Jesula St Fleur
745,sylvietest,sylvie rousseau


## MERGE (Vlookup)

In [38]:
performanceAgents = agent_name.merge(agent,on="username",how="inner")

In [39]:
performanceAgents.to_excel("performancedesAgents.xlsx",index=False,na_rep="NULL")

In [40]:
performanceAgents

,username,name,nbre,organisation,entry_date
0,1yvrose,Yvrose Bertrand,22,CARIS,2021-06-25 05:45:47
1,2ernst,Ernst Belabre,43,CARIS,2021-06-24 08:11:54
2,6francenel,Francenel Dessalines,30,CARIS,2021-06-23 15:40:37
3,6frazyouseline,Fraz-Youseline Dorcent,2,CARIS,2021-06-28 09:02:32
4,6lovemyschinider,Lovemy Schinider Brutus,28,CARIS,2021-06-25 19:21:23
5,6paulina,Paulina Larose,85,CARIS,2021-06-23 12:59:38


In [41]:
performanceAgents.nbre.sum()

210